# Visualizing fire scars through false color

Author: Michelle Yiv

[Link to GitHub Repository](https://github.com/mmyiv/eds220-hwk4)

## About this notebook

Dataset description:

Purpose:
    
Analysis highlight list:
    
References to datasets:

In [4]:
# Import libraries
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import rioxarray as rioxr

In [11]:
# Create filepath to import landsat data
fp = os.path.join('/',
                  'courses',
                  'EDS220',
                  'data',
                  'hwk4_landsat_data',
                  'landsat8-2018-01-26-sb-simplified.nc')
landsat = rioxr.open_rasterio(fp)

# View data
landsat

<xarray.Dataset> Size: 25MB
Dimensions:      (band: 1, x: 870, y: 731)
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 7kB 1.213e+05 1.216e+05 ... 3.557e+05 3.559e+05
  * y            (y) float64 6kB 3.952e+06 3.952e+06 ... 3.756e+06 3.755e+06
    spatial_ref  int64 8B 0
Data variables:
    red          (band, y, x) float64 5MB ...
    green        (band, y, x) float64 5MB ...
    blue         (band, y, x) float64 5MB ...
    nir08        (band, y, x) float64 5MB ...
    swir22       (band, y, x) float64 5MB ...

In [12]:
# Data exploration

Exploration summary:
- Description of variables and dimensions

## True color image

In [18]:
# Drop `band` dimension by `squeeze()` and `drop_vars()`

In [19]:
# Select red, green, and blue and convert to numpy.array using `to_array()`

In [20]:
# Plot data using `.plot.imshow()`

In [21]:
# Adjust band scale to get true color image via robust parameter

Explanation of outputs between part a and b

## False color image

In [25]:
# Make false color by plotting swir22, NIR, and red

## Map

In [27]:
# Map of SWIR/NIR/RED with Thomas fire + figure description